## Batch X-Validation in Python/Scikit-Learn

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import LeaveOneGroupOut, cross_val_score
from sklearn.linear_model import LogisticRegression

In [2]:
datos = pd.read_csv('/Users/carloseqa/Dropbox/RStudioProjects/PredictiveAnalytics/credito.csv')
datos.head()

,id,prestamo,negativo,edad,ingreso,casapropia,noasalariado,dependientes,mesesdir,otrastc,batch
0,1,si,0,37.66667,4.5200,1,0,3,54,12,2
1,2,si,0,33.25000,2.4200,0,0,3,34,13,3
2,3,si,0,33.66667,4.5000,1,0,4,58,5,4
3,4,si,0,30.50000,2.5400,0,0,0,25,7,1
4,5,si,0,32.16667,9.7867,1,0,2,64,5,2


In [3]:
datos=pd.get_dummies(datos,drop_first=True)
datos.head()

,id,negativo,edad,ingreso,casapropia,noasalariado,dependientes,mesesdir,otrastc,batch,prestamo_si
0,1,0,37.66667,4.5200,1,0,3,54,12,2,1
1,2,0,33.25000,2.4200,0,0,3,34,13,3,1
2,3,0,33.66667,4.5000,1,0,4,58,5,4,1
3,4,0,30.50000,2.5400,0,0,0,25,7,1,1
4,5,0,32.16667,9.7867,1,0,2,64,5,2,1


In [4]:
y = datos.prestamo_si
X = datos.drop(datos.columns[[0,9,10]],axis=1)
X.columns

Index(['negativo', 'edad', 'ingreso', 'casapropia', 'noasalariado',
       'dependientes', 'mesesdir', 'otrastc'],
      dtype='object')

Now instanciate the estimator 

In [5]:
logreg1 = LogisticRegression()
logreg1.fit(X,y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [6]:
# print coefficients
logreg1.coef_ , logreg1.intercept_

(array([[-1.43109416, -0.01612337,  0.21449957, -0.22970404, -1.06550785,
         -0.49840537,  0.00205935,  0.15625082]]), array([0.99249965]))

Now estimate accuracy by X-validation 

In [7]:
scores1 = cross_val_score(logreg1,X,y,cv=4) 
scores1.mean(),scores1.std()

(0.7701719576719577, 0.04976055280870805)

Now set up the Batch X-validation 

In [8]:
indice = datos.batch
lolo=LeaveOneGroupOut().split(X, y, indice)

In [9]:
logreg2 = LogisticRegression(C=1e20,penalty='l2')
logreg2.fit(X,y)

LogisticRegression(C=1e+20, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [10]:
# print coefficients
logreg2.coef_ , logreg2.intercept_

(array([[-1.8645436 , -0.05213667,  0.27754191, -0.37192381, -2.64243274,
         -0.55124325,  0.00356787,  0.15866068]]), array([2.27568231]))

In [11]:
scores2 = cross_val_score(logreg2,X,y,cv=lolo) 
scores2.mean(),scores2.std()

(0.7896825396825398, 0.05080122849119449)